<a href="https://colab.research.google.com/github/JuniorHZ19/ML/blob/main/convolucion2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown # *0) ** Instalando libreria(OBLIGATORIO)

!pip install torch torchvision torchaudio

In [ ]:
#@markdown # *1) ** Descargando DataSet  de perros y gatos(OBLIGATORIO)
import zipfile

nombre_zip="data_set_dogcat.zip"

directorio_destino="/content/"

!gdown --id 19u9sy2053Ds739lyYSJut6S_k54trLnd -O {nombre_zip}

with zipfile.ZipFile(nombre_zip, 'r') as zip_ref:
    zip_ref.extractall(directorio_destino)



In [24]:
#@markdown # *1) ** Creacion de el csv de los path y clases(OBLIGATORIO)
import os
import pandas as pd
def crear_paths_csv(directorio_base,clases_lista,clases,nombre_archivo):

   columnas=["path","etiqueta"]
   datos_csv=[]

   for dir_base,dir,clase in  zip(directorio_base,clases_lista,clases):

    for path  in (dir):

     datos_csv.append([dir_base+path ,clase])


   df_lista=pd.DataFrame(datos_csv,columns=columnas)
   df_lista.to_csv(nombre_archivo,index=False)
   print("Csv Creado")

directorio_gato="/content/PetImages/Cat/"
directorio_perro="/content/PetImages/Dog/"

gato_lista=os.listdir(directorio_gato)
perro_list=os.listdir(directorio_perro)

crear_paths_csv([directorio_gato,directorio_perro],[gato_lista,perro_list],[0,1],"Data_set_paths.csv")


Csv Creado


In [155]:
#@markdown # *2) ** Creacion de clase DATASET(OBLIGATORIO)
import torch
from torch.utils.data import Dataset,DataLoader
import pandas as pd

import sklearn
import cv2
from PIL import Image


class MiDataSet(Dataset):

  def __init__(self,csv_file, transform=None):


     self.data=pd.read_csv(csv_file)

     self.x=self.data["path"]
     self.y=self.data["etiqueta"]

     self.transform=transform

     self.samples=self.data["path"].shape[0]


  def __getitem__(self,id):

    rut_imagen=self.x[id]
    imagen=cv2.imread(rut_imagen)

    imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
    etiqueta=self.y[id]

    if self.transform:
      imagen_rgb = self.transform(imagen_rgb)

    return imagen_rgb,etiqueta


  def __len__(self):
   return self.samples





In [139]:

#@markdown # *3) ** Separacion dos archivos difentes para entrenamietno y prueba(OBLIGATORIO)

from sklearn.model_selection import train_test_split

misdatos=pd.read_csv("/content/Data_set_paths.csv")
# Separo mis datos en entrenamiento y test
datos_entrenamiento, datos_evaluacion = train_test_split(misdatos, test_size=0.3, random_state=42)


# Guardar los dos conjuntos en archivos CSV separados
datos_entrenamiento.to_csv("/content/datos_entrenamiento.csv", index=False)
datos_evaluacion.to_csv("/content/datos_evaluacion.csv", index=False)




In [ ]:
#@markdown # *2) ** Mostrar 20 muestras(Opcional)
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from PIL import Image


# Definir transformaciones
transformaciones = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((100, 100))
])

# Crear dataset y DataLoader
muestra = MiDataSet("/content/datos_entrenamiento.csv", transformaciones)
data_muestra = DataLoader(muestra, batch_size=20, shuffle=True)

# Obtener el primer lote
primer_lote = next(iter(data_muestra))
tensor_imagenes, tensor_etiquetas = primer_lote

# Mapeo de etiquetas numéricas a etiquetas de texto
mapeo_etiquetas = {0: 'gato', 1: 'perro'}
etiquetas_texto = [mapeo_etiquetas[item.item()] for item in tensor_etiquetas]

# Crear una figura y subgráficos
fig, axs = plt.subplots(4, 5, figsize=(20, 20))

# Iterar sobre las imágenes del lote
for i in range(4):
    for j in range(5):
        idx = i * 5 + j
        imagen = tensor_imagenes[idx].permute(1, 2, 0)
        etiqueta_texto = etiquetas_texto[idx]

        # Mostrar la imagen en el subgráfico correspondiente
        axs[i, j].imshow(imagen)
        axs[i, j].set_title(etiqueta_texto)
        axs[i, j].axis('off')

plt.show()


In [206]:
import sklearn

from torch.utils.data import TensorDataset

transformaciones = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((100, 100))
])

mis_datos_entrenamiento=MiDataSet("/content/datos_entrenamiento.csv",transform=transformaciones)

mis_datos_test=MiDataSet("/content/datos_evaluacion.csv",transform=transformaciones)







In [ ]:



from torch.utils.data import DataLoader, TensorDataset



data_por_lote_entrenamiento=DataLoader(mis_datos_entrenamiento,batch_size=100,shuffle=True)

data_por_por_lote_test=DataLoader(mis_datos_test,batch_size=100,shuffle=True)



datos_features_train,datos_label_train=next(iter(data_por_lote_entrenamiento))

datos_features_test,datos_label_test=next(iter(data_por_por_lote_test))


print(datos_features_test.shape)




torch.Size([100, 10])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F



class PerceptronMulticapa(nn.Module):

   def __init__(self,input_size):
    super(PerceptronMulticapa,self).__init__()

    self.capa_oculta1=nn.Linear(input_size,3)

    self.capa_oculta2=nn.Linear(3,4)

    self.capa_salida =nn.Linear(4,1)

   def forward(self,x):

     x=self.capa_oculta1(x)
     x=F.relu(x)

     x=self.capa_oculta2(x)
     x=F.relu(x)

     x=self.capa_salida(x)
     output=torch.sigmoid(x)




     return output


In [ ]:



#funcion clasificador:
def clasificador(y_pred_test):

  umbral = 0.6
  y_pred_test_binario = (y_pred_test >= umbral).to(torch.float32)

  return y_pred_test_binario

#funcion acutity:
def accurity(y_test,y_real):
   correctos=0
   for predicho, real in zip(y_test, y_real):
    if predicho==real:
     correctos+=1
   return correctos/len(y_test)


import torch.optim as optim

input_size=X_entrenamiento.shape[1]

PerceptronMulti=PerceptronMulticapa(input_size)

criterio=nn.BCELoss()

optimizer=optim.SGD(PerceptronMulti.parameters(),lr=0.1)

epochs=500

for epoch in range(epochs):

  y_predict=PerceptronMulti(X_entrenamiento)

  loss=criterio(y_predict,y_entrenamiento.view(-1,1))

  loss.backward()

  optimizer.step()

  optimizer.zero_grad()

  with torch.no_grad():
   y_predict_test=PerceptronMulti(X_prueba)
   y_predic_test_clasificado=clasificador(y_predict_test)



   Precision=accurity(y_prueba,y_predic_test_clasificado)



  print(f'Epoca[{epoch+1}],Perdida: {loss.item():.4f} ,Accuracy: {Precision:.4f}')


#nuevo datos:

def escalarNuevaData(data):

    scaler=StandardScaler()

    datos_escalados=scaler.fit_transform(data)

    datos_x_tensor=torch.from_numpy(datos_escalados).float()

    return datos_x_tensor

nuevo_x=escalarNuevaData([[1,29,0,0,211,1,0,0,0,1]])

y_predict=PerceptronMulti(nuevo_x)

print(clasificador(y_predict))

Epoca[1],Perdida: 0.7630 ,Accuracy: 0.6145
Epoca[2],Perdida: 0.7559 ,Accuracy: 0.6107
Epoca[3],Perdida: 0.7493 ,Accuracy: 0.6069
Epoca[4],Perdida: 0.7433 ,Accuracy: 0.6183
Epoca[5],Perdida: 0.7377 ,Accuracy: 0.6183
Epoca[6],Perdida: 0.7325 ,Accuracy: 0.6183
Epoca[7],Perdida: 0.7277 ,Accuracy: 0.6183
Epoca[8],Perdida: 0.7233 ,Accuracy: 0.6183
Epoca[9],Perdida: 0.7191 ,Accuracy: 0.6183
Epoca[10],Perdida: 0.7153 ,Accuracy: 0.6183
Epoca[11],Perdida: 0.7118 ,Accuracy: 0.6183
Epoca[12],Perdida: 0.7084 ,Accuracy: 0.6183
Epoca[13],Perdida: 0.7053 ,Accuracy: 0.6183
Epoca[14],Perdida: 0.7025 ,Accuracy: 0.6183
Epoca[15],Perdida: 0.6998 ,Accuracy: 0.6183
Epoca[16],Perdida: 0.6973 ,Accuracy: 0.6183
Epoca[17],Perdida: 0.6949 ,Accuracy: 0.6183
Epoca[18],Perdida: 0.6927 ,Accuracy: 0.6183
Epoca[19],Perdida: 0.6906 ,Accuracy: 0.6183
Epoca[20],Perdida: 0.6887 ,Accuracy: 0.6183
Epoca[21],Perdida: 0.6869 ,Accuracy: 0.6183
Epoca[22],Perdida: 0.6851 ,Accuracy: 0.6183
Epoca[23],Perdida: 0.6835 ,Accuracy: 0.61

In [ ]:


y_pred_multiclases = torch.tensor([[0.1, 1.8, 0.8, 0.4, 2],
                                   [0.5, 2, 3.5, 5, 2.1]])

def clasificador(y_pred_test):

   max_values_tensor = torch.empty((0,), dtype=torch.float32)

   for tensor in y_pred_test:

     max_values, max_indices  = torch.max(tensor, dim=0)

     max_values_tensor = torch.cat((max_values_tensor, max_indices.unsqueeze(0)))

   return max_values_tensor

print(clasificador(y_pred_multiclases))


tensor([4., 3.])
